# Melbourne House Prices Prediction using XGBoost with tuned parameters for accurate results - itsdylantan

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading and setting up data
> ## Code below to set up data is referenced/based on Alexis Cook's notebook

In [ ]:
import pandas as pd
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

X = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv', index_col='Id')
X_test_full = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv', index_col='Id')

# Drop NA
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Split data 80% 20% train test
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Getting the numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]


my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encoding
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

# Model 1 Build and predictions
> ## Model build using Pipelines

In [ ]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline1 = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', XGBRegressor(random_state= 2001))
])
# Using XGBoost Ensemble Method
my_pipeline1.fit(X_train, y_train)

> # Model predictions and validation


In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

predictions_1 = my_pipeline1.predict(X_valid)
mae_1 = mean_absolute_error(y_valid, predictions_1)
print("Model 1: Mean absolute error = ", mae_1)

# Model 2 (With parameter tuning)
> ## Increasing n_estimators and reducing learning_rate (overfitting prevention)

In [ ]:
my_pipeline2 = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', XGBRegressor(n_estimators=100, learning_rate = 0.05, random_state= 2001))])
# Using XGBoost Ensemble Method
my_pipeline2.fit(X_train, y_train)

predictions_2 = my_pipeline2.predict(X_valid)
mae_2 = mean_absolute_error(y_valid, predictions_2)
print("Model 2: Mean absolute error = ", mae_2)

# Model 1 vs Model 2
**Model 1: Mean absolute error =  17620.33422517123**

**Model 2: Mean absolute error =  17543.9626498288**


In [ ]:
# output = pd.DataFrame({'Id': X_test.index,
#                        'SalePrice': predictions_2})
# output.to_csv('submission.csv', index=False)